In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Conv2D, Input, Reshape, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [ ]:
def build_generator():
  noise_shape = (100,)

  model = Sequential()

  model.add(Dense(256, input_shape = noise_shape))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(BatchNormalization(momentum = 0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(BatchNormalization(momentum = 0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(BatchNormalization(momentum = 0.8))

  model.add(Dense(np.prod(img_shape), activation="tanh"))
  model.add(Reshape(img_shape))

  model.summary()

  noise = Input(shape = noise_shape)

  img = model(noise)

  return Model(noise, img)

In [ ]:
def build_discriminator():

  model = Sequential()

  model.add(Flatten(input_shape = img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Dense(1, activation = "sigmoid"))
  model.summary()

  img = Input(shape = img_shape)
  validity = model(img)

  return Model(img, validity)

In [ ]:
def train(epochs, batch_size = 128, save_interval = 50):
  (X_train, _), (_, _) = mnist.load_data()

  X_train = (X_train.astype(np.float32) - 127.5) / 127.5

  X_train = np.expand_dims(X_train, axis = 3)

  half_batch = int(batch_size/2)


  for epoch in range(epochs):

    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    noise = np.random.normal(0, 1, (half_batch, 100))

    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, 100))

    valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

    g_loss = combined.train_on_batch(noise, valid_y)

    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % save_interval == 0:
        save_imgs(epoch)


In [ ]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/images/mnist_%d.png" % epoch)
    plt.close()


In [ ]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.


In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

In [ ]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [ ]:
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)


In [ ]:
discriminator.trainable = False  


In [ ]:
valid = discriminator(img)  #Validity check on the generated image


In [ ]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


In [ ]:
train(epochs=10000, batch_size=64, save_interval=1000)



0 [D loss: 0.384641, acc.: 67.19%] [G loss: 0.640859]
1 [D loss: 0.343348, acc.: 78.12%] [G loss: 0.669839]
2 [D loss: 0.349203, acc.: 79.69%] [G loss: 0.749374]
3 [D loss: 0.336630, acc.: 84.38%] [G loss: 0.914346]
4 [D loss: 0.332885, acc.: 81.25%] [G loss: 0.997915]
5 [D loss: 0.267296, acc.: 96.88%] [G loss: 1.174884]
6 [D loss: 0.232231, acc.: 96.88%] [G loss: 1.307241]
7 [D loss: 0.205228, acc.: 98.44%] [G loss: 1.568551]
8 [D loss: 0.168442, acc.: 100.00%] [G loss: 1.680493]
9 [D loss: 0.146837, acc.: 100.00%] [G loss: 1.801402]
10 [D loss: 0.114012, acc.: 100.00%] [G loss: 1.979344]
11 [D loss: 0.087923, acc.: 100.00%] [G loss: 2.151517]
12 [D loss: 0.084572, acc.: 100.00%] [G loss: 2.270905]
13 [D loss: 0.073086, acc.: 100.00%] [G loss: 2.309181]
14 [D loss: 0.068509, acc.: 100.00%] [G loss: 2.384034]
15 [D loss: 0.058545, acc.: 100.00%] [G loss: 2.513639]
16 [D loss: 0.059708, acc.: 100.00%] [G loss: 2.532058]
17 [D loss: 0.060189, acc.: 100.00%] [G loss: 2.612051]
18 [D loss

KeyboardInterrupt: ignored